In [0]:
from random import sample

In [0]:
data = sc.textFile("dbfs:/FileStore/FriendDataset.txt")

In [0]:
def parse(line):
    parts = line.split('\t')
    user = parts[0]
    friends = parts[1].split(',') if len(parts) > 1 else []
    return user, friends

In [0]:
parsed = data.map(parse)

In [0]:
def generate_pairs(user, friends):
    for friend in friends:
        yield (user, friend)
    for i in range(len(friends)):
        for j in range(i+1, len(friends)):
            yield ((friends[i], friends[j]), user)
            yield ((friends[j], friends[i]), user)

In [0]:
pairs = parsed.flatMap(lambda x: generate_pairs(x[0], x[1]))

In [0]:
mutuals = pairs.filter(lambda x: isinstance(x[0], tuple)) \
               .groupByKey() \
               .mapValues(lambda x: list(set(x)))

In [0]:
mutual_counts = mutuals.map(lambda x: (x[0][0], (x[0][1], len(x[1]))))

In [0]:
user_friends = parsed.mapValues(set)

In [0]:
def filter_existing_recs(user, recs, friends):
    return [(user, (other, count)) for other, count in recs if other not in friends and user != other]

In [0]:
recs_grouped = mutual_counts.groupByKey().mapValues(list)
joined = recs_grouped.join(user_friends)
filtered_recs = joined.flatMap(lambda x: filter_existing_recs(x[0], x[1][0], x[1][1]))

In [0]:
top10_recs = filtered_recs.groupByKey().mapValues(lambda x: sorted(x, key=lambda y: -y[1])[:10])

In [0]:
results = top10_recs.takeSample(False, 10)
for user, recs in results:
    print(f"{user}\t{','.join([r[0] for r in recs])}")

30577	30585,30561,30595,30565,30548,30571,7246,30556,30542,30551
27942	27940,27974,27920,27934,27927,27970,27944,27966,27972,47235
15717	15114,15736,15113,15712,15707,4518,15755,15765,22945,15721
42738	42703,42720,42749,42706,21078,12333,31982,42709,42727,31989
757	36559,697,778,11359,7437,24219,14442,93,29637,825
32108	24951,8695,46505,20161,20156,11380,5957,7406,20154,44570
11587	2103,11562,11544,11588,11557,28112,11543,42206,4101,2050
32826	32749,32837,32745,6958,32835,8862,34226,8742,32848,17504
44587	44581,44576,44572,44577,44582,44579,44588,2409,9997,44573
959	966,957,1023,990,975,971,44359,1013,995,33023
